In [ ]:
!pip install PySDNSim

In [ ]:
!sudo apt-get install python dash

In [ ]:
!sudo pip install dash==0.29.0

In [ ]:
!sudo python -m pip install dash==0.29.0

In [ ]:
!pip install jupyter-dash

In [ ]:
!pip install --upgrade pip wheel setuptools requests

In [ ]:
!python3 -m pip install dash

In [ ]:
!pip install jupyter-dash

In [ ]:
from collections.abc import MutableMapping

In [ ]:
!python3 -m pip install dash

In [ ]:
!pip install --upgrade dash dash-core-components dash-html-components dash-renderer

  Using cached dash-2.15.0-py3-none-any.whl.metadata (11 kB)
Using cached dash-2.15.0-py3-none-any.whl (10.2 MB)
  Attempting uninstall: dash
    Found existing installation: dash 0.29.0
    Uninstalling dash-0.29.0:
      Successfully uninstalled dash-0.29.0


In [ ]:
!pip uninstall dash

Found existing installation: dash 2.15.0
Uninstalling dash-2.15.0:
  Would remove:
    /usr/local/bin/dash-generate-components
    /usr/local/bin/dash-update-components
    /usr/local/bin/renderer
    /usr/local/etc/jupyter/nbconfig/notebook.d/dash.json
    /usr/local/lib/python3.10/dist-packages/dash-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/dash/*
    /usr/local/share/jupyter/lab/extensions/dash-jupyterlab.tgz
    /usr/local/share/jupyter/nbextensions/dash/main.js
Proceed (Y/n)?   ERROR: Operation cancelled by user


In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 18.8 MB/s eta 0:00:00


In [ ]:
#import dash
from dash import Dash, Input, Output, dcc, html

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

ModuleNotFoundError: No module named 'jupyter_dash'

In [ ]:
!pip install treelib

In [ ]:
import random
from copy import deepcopy
from typing import List
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
#import jupyter-dash
from dash import Dash, Input, Output, dcc, html
from PySDNSim.Backend import Backend
from PySDNSim.Config import Config
from PySDNSim.Experiment import Experiment
from PySDNSim.Host import Host
from PySDNSim.Job import Job
from PySDNSim.Microservice import Microservice
from PySDNSim.NetworkService import NetworkService
from PySDNSim.NetworkService import create_network_service
from treelib import Node, Tree


# create simulation config

In [ ]:
sim_config = Config(seed=1024, interval=1.0, step_size=0.01)

# create host

In [ ]:
host = Host(
    cpus=64,
    ram=102400,
    bw=100000,
    storage=1024000,
    max_power=1600.0,
    static_power=300.0,
    replicas=3,
)

# Create Microservices

In [ ]:
microservices: List[Microservice] = list()
ms_mqtt_broker = Microservice(
    name="mqtt_broker",
    size=512,
    cpus=2,
    replicas=1,
    max_replicas=3,
    cpu_ratio=25,
    ram_ratio=32,
    bw_ratio=25,
)
ms_mqtt_broker.add_auto_scale("cpu",0.5)
ms_mqtt_broker.add_auto_scale("ram",0.5)
ms_mqtt_broker.add_auto_scale("bw",0.5)
microservices.append(ms_mqtt_broker)
ms_chirpstack_gateway = Microservice(
    name="chirpstack_gateway",
    size=128,
    cpus=2,
    replicas=1,
    max_replicas=3,
    cpu_ratio=25,
    ram_ratio=32,
    bw_ratio=25,
)
ms_chirpstack_gateway.add_auto_scale("cpu",0.5)
ms_chirpstack_gateway.add_auto_scale("ram",0.5)
ms_chirpstack_gateway.add_auto_scale("bw",0.5)
microservices.append(ms_chirpstack_gateway)
ms_chirpstack = Microservice(
    name="chirpstack",
    size=128,
    cpus=4,
    replicas=1,
    max_replicas=3,
    cpu_ratio=10,
    ram_ratio=32,
    bw_ratio=25,
)
ms_chirpstack.add_auto_scale("cpu",0.5)
ms_chirpstack.add_auto_scale("ram",0.5)
ms_chirpstack.add_auto_scale("bw",0.5)
microservices.append(ms_chirpstack)
ms_chirpstack_rest_api = Microservice(
    name="chirpstack_rest_api",
    size=128,
    cpus=2,
    replicas=1,
    max_replicas=3,
    cpu_ratio=5,
    ram_ratio=128,
    bw_ratio=25,
)
ms_chirpstack_rest_api.add_auto_scale("cpu",0.5)
ms_chirpstack_rest_api.add_auto_scale("ram",0.5)
ms_chirpstack_rest_api.add_auto_scale("bw",0.5)
microservices.append(ms_chirpstack_rest_api)
ms_postgresql = Microservice(
    name="postgresql",
    size=2048,
    cpus=2,
    replicas=1,
    max_replicas=3,
    cpu_ratio=50,
    ram_ratio=128,
    bw_ratio=100,
)
ms_postgresql.add_auto_scale("cpu",0.5)
ms_postgresql.add_auto_scale("ram",0.5)
ms_postgresql.add_auto_scale("bw",0.5)
microservices.append(ms_postgresql)
ms_redis = Microservice(
    name="redis",
    size=2048,
    cpus=2,
    replicas=1,
    max_replicas=3,
    cpu_ratio=50,
    ram_ratio=128,
    bw_ratio=100,
)
ms_redis.add_auto_scale("cpu",0.5)
ms_redis.add_auto_scale("ram",0.5)
ms_redis.add_auto_scale("bw",0.5)
microservices.append(ms_redis)

# Create network services


In [ ]:
ns_list = list()
register_device = create_network_service(
    name="register_device",
    microservices=["chirpstack", "redis", "chirpstack_gateway"],
    schdeule=[0, 1, 1],
    schedule_length=[10, 10, 10],
    ms_pool=microservices
)
ns_list.append(register_device)
read_data = create_network_service(
    name="receive_data",
    microservices=["chirpstack_gateway", "mqtt_broker", "chirpstack", "postgresql"],
    schdeule=[0, 1, 2, 3],
    schedule_length=[10, 10, 10, 10],
    ms_pool=microservices
)
ns_list.append(read_data)
retrive_data = create_network_service(
    name="retrive_data",
    microservices=["chirpstack_rest_api", "chirpstack", "postgresql", "chirpstack"],
    schdeule=[0, 1, 2, 3],
    schedule_length=[10, 10, 10, 10],
    ms_pool=microservices
)
ns_list.append(retrive_data)

# Start a simulation that randomly select network services at each iteration.

In [ ]:
from PySDNSim.Backend import Backend

ModuleNotFoundError: No module named 'PySDNSim'

In [ ]:
import os
os.system('start backend.jar')

32512

In [ ]:
import subprocess
subprocess.call(['java', '-jar', 'backend.jar'])

In [ ]:
for iter in range(10):
    num_ns = random.randint(1, 20)
    chosen_ns: List[NetworkService] = deepcopy(random.choices(ns_list, k=num_ns))
    for ns in chosen_ns:
        ns.offset_schedule(random.randint(0, 5))
    experiment = Experiment(
        name=f"{iter}",
        config=sim_config,
        host=host,
        microservices=microservices,
        network_services=chosen_ns,
    )
    bk = Backend()
    bk.run_experiment(experiment=experiment, output_path="results2")

    #Backend.run_experiment(self, experiment=experiment, output_path="./results")
    #Backend.run_experiment(self, experiment=experiment, output_path="./results")

In [ ]:
import subprocess
subprocess.call(
                        [
                            "java",
                            "-jar",
                            "backend.jar",
                            "./configs/0.json",
                            "./results/0",
                        ]
                    )

1

In [ ]:
!java -version

openjdk version "11.0.21" 2023-10-17
OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [ ]:
import subprocess
cmd="java -jar backend.jar edu.stanford.nlp.ie.crf.CRFClassifier -prop prop.txt"
output = subprocess.check_output(cmd)

In [ ]:
import cv2
import numpy as np
import os
from glob import glob
from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn import preprocessing


data = []

# Read images from file
for filename in glob('Images/*.jpg'):

    img = cv2.imread(filename)
    height, width = img.shape[:2]
    img = np.array(img)

    # Check that all my images are of the same resolution
    if height == 529 and width == 940:

        # Reshape each image so that it is stored in one line
        img = np.concatenate(img, axis=0)
        img = np.concatenate(img, axis=0)
        data.append(img)

# Normalise data
data = np.array(data)
Norm = preprocessing.Normalizer()
Norm.fit(data)
data = Norm.transform(data)

# PCA model
pca = PCA(0.95)
pca.fit(data)
data = pca.transform(data)

# K-Nearest neighbours
knn = neighbors.NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric='minkowski').fit(data)
distances, indices = knn.kneighbors(data)

print(indices)